In [ ]:
import pandas as pd
import random

DailyData = []
TestData = []

# Define the range for each column
range_data = [(0, 5000), (0, 5000), (0, 20), (0, 10)]

# Set the number of additional data points
num_additional_data = 500

# Generate and add additional data
for _ in range(num_additional_data):
    additional_data = [random.randint(range_data[i][0], range_data[i][1]) for i in range(len(range_data))]
    DailyData.append(additional_data)

for _ in range(num_additional_data):
    additional_data = [random.randint(range_data[i][0], range_data[i][1]) for i in range(len(range_data))]
    TestData.append(additional_data)

# Print the updated data
for data in DailyData:
    print(data)

for data in TestData:
    print(data)

Columns = ['Number of Restock', 'Number of sold items', 'Number of ADS', 'Number of Visitors']
Data = pd.DataFrame(DailyData, columns=Columns)
Data_t = pd.DataFrame(TestData, columns=Columns)

In [26]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the neural network model
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Set the hyperparameters
input_size = 2  # 광고 수, 방문자 수
hidden_size = 8
output_size = 1
learning_rate = 0.01
num_epochs = 10
batch_size = 32

X_train = torch.tensor(Data[['Number of ADS', 'Number of Visitors']].values, dtype=torch.float32)
y_train = torch.tensor(Data['Number of Visitors'].values, dtype=torch.float32)
X_test = torch.tensor(Data_t[['Number of ADS', 'Number of Visitors']].values, dtype=torch.float32)
y_test = torch.tensor(Data_t['Number of Visitors'].values, dtype=torch.float32)

# Create the neural network model
model = NeuralNetwork(input_size, hidden_size, output_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    # Mini-batch training
    for i in range(0, len(X_train), batch_size):
        # Prepare the mini-batch
        inputs = X_train[i:i+batch_size]
        targets = y_train[i:i+batch_size]

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print the loss for every epoch
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

# Evaluation
model.eval()
with torch.no_grad():
    # Forward pass on test data
    test_outputs = model(X_test)
    test_predictions = test_outputs.round()  # 반올림하여 예측값 생성
    accuracy = (test_predictions == y_test).float().mean()
    print(f'Test Accuracy: {accuracy.item()}')

Epoch 1/10, Loss: 11.101415634155273
Epoch 2/10, Loss: 9.389911651611328
Epoch 3/10, Loss: 9.194513320922852
Epoch 4/10, Loss: 9.049145698547363
Epoch 5/10, Loss: 8.930463790893555
Epoch 6/10, Loss: 8.837577819824219
Epoch 7/10, Loss: 8.761648178100586
Epoch 8/10, Loss: 8.703571319580078
Epoch 9/10, Loss: 8.654668807983398
Epoch 10/10, Loss: 8.626051902770996
Test Accuracy: 0.08904000371694565


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([20])) that is different to the input size (torch.Size([20, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
